In [17]:
import os
import json
from PIL import Image

In [25]:
# 입력 데이터 경로
type = "val"
label_dir = f"./door_image_set/{type}/labels"  # 기존 라벨 파일 디렉토리
image_dir = f"./door_image_set/{type}/images"  # 이미지 파일 디렉토리
output_file = f"{type}_coco_annotations.json"  # COCO JSON 파일 이름

# 클래스 ID 매핑
class_id_mapping = {"/m/02dgv": 81}  # Door 클래스 ID를 숫자로 매핑

# COCO 데이터 구조 초기화
coco_data = {
    "images": [],
    "annotations": [],
    "categories": [{"id": 81, "name": "door"}]  # Door 클래스 추가
}

# # 클래스 정보 추가
# for class_name, class_id in class_id_mapping.items():
#     coco_data["categories"].append({
#         "id": class_id,
#         "name": class_name.split("/")[-1]
#     })

# ID 초기화
image_id = 0
annotation_id = 0

In [26]:
# 이미지와 라벨 데이터 변환
for label_file in os.listdir(label_dir):
    # 이미지 파일과 매칭
    image_file = label_file.replace(".txt", ".jpg")
    image_path = os.path.join(image_dir, image_file)

    # (Pillow 사용) 이미지 크기 확인
    with Image.open(image_path) as img:
        width, height = img.size

    # COCO 이미지 정보 추가
    image_id += 1
    coco_data["images"].append({
        "id": image_id,
        "file_name": image_file,
        "width": width,
        "height": height
    })

    # 라벨 파일 읽기 및 COCO Annotation 추가
    label_path = os.path.join(label_dir, label_file)
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            class_name = parts[0]
            x_min, y_min, width_ratio, height_ratio = map(float, parts[1:])

            # 클래스 ID 변환
            category_id = class_id_mapping.get(class_name, -1)
            if category_id == -1:
                print(f"Unknown class: {class_name}")
                continue

            # Bounding Box 절대 좌표 계산
            bbox = [
                x_min * width,                # x_min 절대 좌표
                y_min * height,               # y_min 절대 좌표
                width_ratio * width,          # Bounding Box 너비
                height_ratio * height         # Bounding Box 높이
            ]

            # COCO Annotation 추가
            annotation_id += 1
            coco_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": bbox,
                "area": bbox[2] * bbox[3],  # 면적 계산
                "iscrowd": 0
            })

In [27]:
# COCO JSON 파일 저장
with open(output_file, "w") as f:
    json.dump(coco_data, f, indent=4)